In [2]:
seed = 12345
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [1]:
import matplotlib 
matplotlib.use('Agg') 
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
import os
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing import image
from keras.models import load_model
import pandas as pd
import h5py
import numpy as np

os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32"

Using Theano backend.
Using gpu device 0: GeForce GTX 960M (CNMeM is disabled, cuDNN not available)


In [4]:
# Load the original trained model with its weights
model = load_model('fine_tuned_model_all.h5')

In [5]:
# Load the fine-tuned weights
model.load_weights('weights.best_28_11.hdf5')

In [6]:
train_data_dir = './UECFOOD100/data/train'
validation_data_dir = './UECFOOD100/data/validation'
img_width, img_height = 150, 150
nb_train_samples = 10183
nb_validation_samples = 4428

In [7]:
test_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=1,
        class_mode='categorical',
        shuffle=False,
        seed=seed)

Found 4428 images belonging to 100 classes.


In [8]:
prediction_to_class_map = {v:k for k,v in validation_generator.class_indices.items()}

# Save the Class-index map in a pickle file:
<code> 
import pickle 
with open('class_indices_map.p', 'wb') as fp:
    pickle.dump(prediction_to_class_map, fp)
</code>
To load it use:
<code>
with open('class_indices_map.p', 'rb') as fp:
    prediction_to_class_map = pickle.load(fp)
</code>

In [9]:
# img_path = './UECFOOD100/raw_data/6/501.jpg'
img_path = './UECFOOD100/data/train/12/1122_0.jpg'
img = image.load_img(img_path, target_size=(150, 150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

clas = model.predict_classes(x/255)[0]

preds = model.predict(x/255)
prediction_to_class_map[clas]
np.vectorize(prediction_to_class_map.get)(preds.argsort()[0,-1:-6:-1])

1/1 [==============================] - 0s


'12'

array(['12', '67', '95', '19', '71'], 
      dtype='<U2')

In [10]:
model.evaluate_generator(validation_generator, val_samples=4428)
# [1.2357008281115918, 0.69489611562782294]

[1.2357007828612154, 0.69489611562782294]

In [116]:
test_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=1,
        class_mode='categorical',
        shuffle=False,
        seed=seed)

predictions = model.predict_generator(validation_generator, val_samples=4428)

Found 4428 images belonging to 100 classes.


In [117]:
sum(predictions[:188].argmax(axis=-1)!=0)

9

In [72]:
val_lab_df = pd.DataFrame(validation_labels)
val_lab_df.columns=['Labels']

In [73]:
val_lab_df

,Labels
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


In [76]:
val_lab_df.groupby('Labels').size()

Labels
0     188
1      41
2      32
3      34
4      68
5      37
6      36
7      31
8      51
9      73
10     41
11     50
12     39
13     47
14     32
15     49
16    106
17     42
18     34
19     40
20     46
21     42
22     41
23     35
24     35
25     36
26     39
27     31
28     31
29     35
     ... 
70     36
71     33
72     33
73     32
74     45
75     34
76     37
77     36
78     30
79     33
80     37
81     38
82     43
83     38
84     35
85     40
86    105
87     33
88     35
89     51
90     36
91     51
92     51
93     34
94     57
95     33
96     41
97     35
98     47
99     43
dtype: int64

In [85]:
12/43

0.27906976744186046

In [113]:
label_gen = ImageDataGenerator()
label_generator = label_gen.flow_from_directory(validation_data_dir,target_size=(2,2),
        batch_size=1,
        class_mode='categorical',
        shuffle=False,
        seed=seed)
i=0
validation_labels= []
for _, yy in label_generator:
    i += 1
    validation_labels = np.hstack([validation_labels,np.argmax(yy)])
    if i >= nb_validation_samples:
        break

validation_labels = validation_labels.astype(int)

Found 4428 images belonging to 100 classes.


In [52]:
# 1351/4428 # 0.30510388437217706

In [ ]:
# def top_k(k=5):
#     #return a set of top k predictions for each data point"""
#     def nested_function(preds):
#         return set(preds[-1:-k-1:-1])
#     return nested_function

In [60]:
# validation_labels in np.apply_along_axis(top_k(5), 1, a)

In [122]:
# def top_5(preds):
#     #return a set of top k predictions for each data point"""
#     return set(preds[-1:-6:-1])    

In [135]:
predictions.argsort(axis=-1)[i]

array([39, 44, 79, 47,  5,  6, 54, 72, 41, 63,  9, 69, 98, 73, 33, 32, 59,
       76, 18, 17, 55, 62, 97, 28, 58, 57,  3, 53, 12, 84,  7, 16, 83, 20,
       42, 26, 61, 82, 21, 70, 11, 27, 52, 67, 46, 71,  4, 93, 19, 77, 49,
       90, 35, 88, 64, 15, 95,  8, 36, 13, 40, 22, 94, 30, 74, 25, 14, 43,
       48, 81, 96, 51, 31, 38, 37, 65, 56, 68,  2, 60, 34, 50, 10, 66, 45,
       92, 91, 29, 80, 75,  1, 78, 24, 87, 89, 86, 23, 85, 99,  0], dtype=int64)

In [136]:
predictions.argsort(axis=-1)[i,:-6:-1]

array([ 0, 99, 85, 23, 86], dtype=int64)

In [126]:
predictions.argsort(axis=-1)[i,[-1,-2,-3,-4,-5]]

array([ 0, 99, 85, 23, 86], dtype=int64)

In [129]:
predictions.argsort(axis=-1)[i,:-5:]

array([39, 44, 79, 47,  5,  6, 54, 72, 41, 63,  9, 69, 98, 73, 33, 32, 59,
       76, 18, 17, 55, 62, 97, 28, 58, 57,  3, 53, 12, 84,  7, 16, 83, 20,
       42, 26, 61, 82, 21, 70, 11, 27, 52, 67, 46, 71,  4, 93, 19, 77, 49,
       90, 35, 88, 64, 15, 95,  8, 36, 13, 40, 22, 94, 30, 74, 25, 14, 43,
       48, 81, 96, 51, 31, 38, 37, 65, 56, 68,  2, 60, 34, 50, 10, 66, 45,
       92, 91, 29, 80, 75,  1, 78, 24, 87, 89], dtype=int64)

In [137]:
correct = 0
for i in range(nb_validation_samples):
    correct += validation_labels[i] in predictions.argsort(axis=-1)[i,[-1,-2,-3,-4,-5]]

In [138]:
top10 = 0
for i in range(nb_validation_samples):
    top10 += validation_labels[i] in predictions.argsort(axis=-1)[i,:-11:-1]

In [139]:
accuracy = correct/nb_validation_samples
accuracy  # 0.912375790424571
top10accuracy = top10/nb_validation_samples
top10accuracy  # 0.9566395663956639

0.912375790424571

0.9566395663956639

In [146]:
correct

4040

In [142]:
4040/4428

0.912375790424571

In [56]:
validation_data_dir = "./UECFOOD100/data/validation/"
d = []
for food_class in os.listdir(validation_data_dir):
    for image_name in os.listdir(os.path.join(validation_data_dir, food_class)):
        if image_name.endswith(".jpg"):
            img_path = os.path.join(validation_data_dir, food_class,image_name)
            img = image.load_img(img_path, target_size=(150, 150))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
#             mapped_class = model.predict_classes(x/255, verbose=0)[0]
#             clas = prediction_to_class_map[mapped_class]
#             print('predicted class is:', clas)
            preds = model.predict(x/255)            
            preds = preds.argsort()[0,-1::-1]
            preds = np.vectorize(prediction_to_class_map.get)(preds)
            d.append({'image': image_name, 'actual_class': food_class, 'predicted_class': preds[0], 'pred_prob': preds})
# #             print('image:', image_name, 'True:', food_class,  'predicted:', preds )
df = pd.DataFrame(d)

In [57]:
errors = df[df['predicted_class']!=df.actual_class]

In [147]:
df[(df.predicted_class == df.actual_class)].shape[0]

3077

In [148]:
3077/df.shape[0]

0.6948961156278229

In [149]:
errors.head()

,actual_class,image,pred_prob,predicted_class
27,1,13264_0.jpg,"[87, 68, 6, 1, 81, 63, 86, 99, 94, 9, 3, 55, 5...",87
33,1,13639_0.jpg,"[77, 4, 95, 5, 10, 76, 12, 99, 88, 64, 50, 2, ...",77
34,1,13671_0.jpg,"[70, 87, 86, 30, 68, 1, 37, 12, 91, 95, 99, 88...",70
137,1,16290_0.jpg,"[37, 1, 91, 6, 70, 83, 53, 68, 82, 89, 90, 43,...",37
147,1,41_0.jpg,"[5, 92, 84, 87, 4, 6, 86, 9, 30, 10, 91, 27, 1...",5


In [302]:
error_ratio = errors.groupby('actual_class').count()['image']/df.groupby('actual_class').count()['image']

In [307]:
error_ratio

actual_class
1      0.047872
10     0.243902
100    0.187500
11     0.117647
12     0.308824
13     0.189189
14     0.111111
15     0.193548
16     0.647059
17     0.082192
18     0.146341
19     0.340000
2           NaN
20     0.127660
21     0.468750
22     0.061224
23     0.094340
24     0.619048
25     0.411765
26     0.275000
27     0.304348
28     0.285714
29     0.317073
3      0.600000
30     0.514286
31     0.583333
32     0.358974
33     0.516129
34     0.193548
35     0.628571
         ...   
72     0.055556
73     0.151515
74     0.484848
75     0.281250
76     0.311111
77     0.529412
78     0.108108
79     0.472222
8      0.333333
80     0.090909
81     0.405405
82     0.289474
83     0.441860
84     0.184211
85     0.314286
86     0.550000
87     0.295238
88     0.333333
89     0.628571
9      0.372549
90     0.750000
91     0.568627
92     0.176471
93     0.323529
94     0.140351
95     0.272727
96     0.024390
97     0.228571
98     0.255319
99     0.604651
Name: image

In [131]:
errors.groupby('actual_class').count()['image']

actual_class
1       9
10     10
100     6
11      4
12     21
13      7
14      4
15      6
16     33
17      6
18      6
19     17
20      6
21     15
22      3
23     10
24     26
25     14
26     11
27     14
28     12
29     13
3      21
30     18
31     21
32     14
33     16
34      6
35     22
36     22
       ..
72      2
73      5
74     16
75      9
76     14
77     18
78      4
79     17
8      10
80      3
81     15
82     11
83     19
84      7
85     11
86     22
87     31
88     11
89     22
9      19
90     27
91     29
92      9
93     11
94      8
95      9
96      1
97      8
98     12
99     26
Name: image, dtype: int64

In [133]:
df.groupby('actual_class').count()[['image']]

,image
actual_class,
1,188
10,41
100,32
11,34
12,68
13,37
14,36
15,31
16,51


In [134]:
errors.pred_prob.get_values()

array([ array(['87', '68', '6', '1', '81', '63', '86', '99', '94', '9', '3', '55',
       '54', '32', '35', '45', '47', '56', '69', '29', '88', '60', '30',
       '70', '85', '8', '83', '27', '31', '16', '15', '12', '26', '10',
       '5', '61', '92', '91', '84', '21', '52', '40', '96', '48', '57',
       '43', '76', '50', '28', '78', '37', '93', '18', '42', '67', '41',
       '77', '33', '62', '22', '39', '82', '79', '24', '59', '4', '7',
       '95', '20', '19', '72', '46', '44', '11', '58', '74', '23', '34',
       '100', '73', '17', '80', '25', '89', '71', '90', '65', '53', '13',
       '14', '98', '51', '97', '75', '64', '38', '66', '49', '36', '2'], 
      dtype='<U3'),
       array(['77', '4', '95', '5', '10', '76', '12', '99', '88', '64', '50', '2',
       '21', '86', '90', '18', '30', '11', '87', '40', '36', '57', '89',
       '54', '92', '70', '58', '31', '35', '33', '8', '100', '26', '16',
       '34', '9', '82', '1', '67', '24', '72', '3', '7', '96', '28', '52',
       '45'

In [284]:
os.listdir(r'C:\Users\ali_a\Dropbox')

['.dropbox',
 '.dropbox.cache',
 '20161130_132750.jpg',
 '20161130_132750_copped.jpg',
 '301.jpg',
 'desktop.ini',
 'IMG_20161208_110617.jpg',
 'Nanbanzuke_comp.jpg',
 'test.jpg']

In [371]:
for f in os.listdir(r'C:\Users\ali_a\Dropbox'):
    if f.endswith(".jpg") and 'prediction' not in f:
        img_path = os.path.join(r'C:\Users\ali_a\Dropbox',f)
        try:
            img = image.load_img(img_path, target_size=(150, 150))             
        except:
            print('file read error. Skipped')
            continue
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        #             mapped_class = model.predict_classes(x/255, verbose=0)[0]
        #             clas = prediction_to_class_map[mapped_class]
        #             print('predicted class is:', clas)
        live_prediction = model.predict(x/255)            
        live_prediction = live_prediction.argsort()[0,-1::-1]
        live_prediction = np.vectorize(prediction_to_class_map.get)(live_prediction)
        print (img_path[:-4]+'_prediction.jpg', live_prediction[0:6])

C:\Users\ali_a\Dropbox\687_prediction.jpg ['7' '97' '16' '38' '17' '74']
C:\Users\ali_a\Dropbox\IMG_20161208_142251_prediction.jpg ['7' '16' '70' '46' '19' '71']
C:\Users\ali_a\Dropbox\IMG_20161208_142358_prediction.jpg ['48' '7' '76' '52' '46' '16']
C:\Users\ali_a\Dropbox\IMG_20161208_142557_prediction.jpg ['16' '48' '7' '52' '76' '46']
C:\Users\ali_a\Dropbox\sushi_prediction.jpg ['7' '71' '81' '75' '67' '15']
C:\Users\ali_a\Dropbox\test_prediction.jpg ['9' '3' '99' '33' '100' '86']


In [298]:
live_prediction

array(['50', '26', '93', '22', '27', '84', '11', '57', '24', '63', '72',
       '70', '96', '33', '88', '52', '92', '58', '28', '69', '23', '20',
       '31', '77', '34', '40', '48', '61', '73', '21', '6', '5', '29',
       '76', '87', '98', '39', '35', '17', '82', '78', '94', '12', '16',
       '64', '9', '83', '60', '49', '68', '62', '19', '42', '75', '4',
       '55', '54', '65', '97', '100', '56', '86', '95', '3', '8', '45',
       '18', '81', '44', '74', '59', '89', '99', '90', '37', '10', '47',
       '2', '30', '53', '91', '7', '13', '41', '46', '67', '1', '85', '71',
       '15', '43', '25', '66', '32', '79', '51', '36', '80', '38', '14'], 
      dtype='<U3')

In [645]:
# from PIL import ImageFont
from PIL import ImageDraw
from PIL import Image
from PIL import ImageFont
# img = Image.open("sample_in.jpg")
img_path = os.path.join(r'C:\Users\ali_a\Dropbox','IMG_20161208_142251.jpg')
img = Image.open(img_path)
# Resize large images:
if img.size[0] > 1000:
    ratio = img.size[0]/1000
    new_width = 1000
    new_height = int(img.size[1]/ratio)
    img = img.resize((new_width, new_height), resample=Image.BILINEAR)

draw = ImageDraw.Draw(img)
fontsize = 1
font = ImageFont.truetype(r'C:\Windows\Fonts\ARIAL.TTF', fontsize)
text = 'Class: ' + str(live_prediction[0:6]) + '\n' + 'Calories: ' + str(521)

while draw.textsize(text, font=font)[0] < 0.3*img.size[0]:
    fontsize +=1
    font = ImageFont.truetype(r'C:\Windows\Fonts\ARIAL.TTF', fontsize)

fontsize -= 1
font = ImageFont.truetype(r'C:\Windows\Fonts\ARIAL.TTF', fontsize)
textsize = draw.textsize(text, font=font)
offset = font.getoffset(text)

draw.rectangle((0,0,textsize[0]+offset[0],textsize[1]+offset[1]), fill=(255,255,255))
draw.text((0, 0),text,(0,0,0),font=font)

output_path = img_path[:-4]+'_prediction.jpg'
img.save(output_path)

In [608]:
textsize = draw.textsize(text, font=font)
textsize

(106, 22)

In [604]:
draw.rectangle((0,0,textsize[0]+offset[0],textsize[1]+offset[1]), fill=(255,255,255))

In [605]:
offset = font.getoffset(text)
offset

(0, 1)

In [606]:
draw.text((0, 0),text,(0,0,0),font=font)

In [490]:
img.size

(288, 200)

In [529]:
font.path

'C:\\Windows\\Fonts\\ARIALUNI.TTF'

In [ ]:
img.info

In [ ]:
!pip uninstall piexif

In [338]:
import piexif
exif_dict = piexif.load(img_path)

In [319]:
# exif_dict['Predicted']=True

In [342]:
# exif_bytes = piexif.dump(exif_dict)
img.save(output_path, exif=exif_bytes)

In [340]:
exif_dict

{'0th': {}, '1st': {}, 'Exif': {}, 'GPS': {}, 'Interop': {}, 'thumbnail': None}

In [336]:
exif_dict = piexif.load(img.info["exif"])

In [337]:
exif_dict

{'0th': {}, '1st': {}, 'Exif': {}, 'GPS': {}, 'Interop': {}, 'thumbnail': None}

In [344]:
img = Image.open(img_path)

In [345]:
img.info

{'dpi': (300, 300),
 'jfif': 257,
 'jfif_density': (300, 300),
 'jfif_unit': 1,
 'jfif_version': (1, 1)}

In [349]:
img.close()

In [359]:
type(img)

PIL.Image.Image

In [360]:
img.close()